In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd
from PIL import Image

In [2]:
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [3]:
# replace images with the image you want to compress
images = {
    "Beautiful Nature": np.asarray(Image.open('nature1.jpg')),
    "Tree in Nature": np.asarray(Image.open('nature2.jpg')),
    "Dog": np.asarray(Image.open('animal1.jpg')),
    "Lion": np.asarray(Image.open('animal2.jpg')),
    "Jackals": np.asarray(Image.open('animal3.jpg')),
    "Flower (white)": np.asarray(Image.open('flower1.jpg')),
    "Flower (coloured)": np.asarray(Image.open('flower2.jpeg')),
    "Rahul Dangi": np.asarray(Image.open('developer.jpg')),
}

In [4]:
def show_images(img_name):
    'It will show image in widgets'
    print("Loading...")
    plt.title("Image Name: "+img_name+"\n")
    plt.imshow(images[img_name])
    plt.axis('off')
    plt.show()

In [5]:
interact(show_images, img_name=list(images.keys()))

interactive(children=(Dropdown(description='img_name', options=('Beautiful Nature', 'Tree in Nature', 'Dog', '…

<function __main__.show_images(img_name)>

In [6]:
compressed_image = None
text = None

In [7]:
def on_save_clicked(b):
    img_name = text.value.split()[0]
    print("saving image...")
    compressed_image.save(img_name+".jpg")
    print("image saved.")
    
    
def compress_image(img_name, k):
    print("processing...")
    global text
    global compressed_image
    img = images[img_name]
    
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    
    print("compressing...")
    ur,sr,vr = svd(r, full_matrices=False)
    ug,sg,vg = svd(g, full_matrices=False)
    ub,sb,vb = svd(b, full_matrices=False)
    rr = np.dot(ur[:,:k],np.dot(np.diag(sr[:k]), vr[:k,:]))
    rg = np.dot(ug[:,:k],np.dot(np.diag(sg[:k]), vg[:k,:]))
    rb = np.dot(ub[:,:k],np.dot(np.diag(sb[:k]), vb[:k,:]))
    
    print("arranging...")
    rimg = np.zeros(img.shape)
    rimg[:,:,0] = rr
    rimg[:,:,1] = rg
    rimg[:,:,2] = rb
    
    for ind1, row in enumerate(rimg):
        for ind2, col in enumerate(row):
            for ind3, value in enumerate(col):
                if value < 0:
                    rimg[ind1,ind2,ind3] = abs(value)
                if value > 255:
                    rimg[ind1,ind2,ind3] = 255

    compressed_image = rimg.astype(np.uint8)
    
    plt.title("Image Name: "+img_name+"\n")
    plt.imshow(compressed_image)
    plt.axis('off')
    plt.show()
    
    save = widgets.Button(description="Save image!", button_style='danger')
    save.on_click(on_save_clicked)
    
    compressed_image = Image.fromarray(compressed_image)
    
    text = widgets.Text(value="image_save",placeholder='Enter image name...',description='Image name')
    display(text)
    display(save)
    

In [8]:
interact(compress_image, img_name=list(images.keys()), k = (0,200))

interactive(children=(Dropdown(description='img_name', options=('Beautiful Nature', 'Tree in Nature', 'Dog', '…

<function __main__.compress_image(img_name, k)>